In [ ]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [ ]:
#load dataset into directory from drive

In [ ]:
xform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()]) #primary transform

According to the second link cited in the proposal, a grayscale transofrm on top of resizing has given the best results when comparing sentiments on faces


In [ ]:
#any possible data augmentation here:






In [ ]:
dataset_full = datasets.ImageFolder('/content/drive/MyDrive/dataset', transform=xform) #retreive dataset, change location 

In [ ]:
n_all = len(dataset_full)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(1549)
dataset_train, dataset_test = torch.utils.data.random_split(dataset_full, [n_train, n_test], rng)
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = 4, shuffle=True) #prepare dataset by splitting, same as usual

In [ ]:
model = models.resnet18(pretrained=True) #will be doing transfer learning on resnet. 

as done in A2, need to modify layers to attain better accuracies later on. could experiment on raw resnet first and modify fc layers later on.

In [ ]:
#code here for running test and model using cross entry loss







#code here for optimizer and scheduler








In [ ]:
# https://ieeexplore.ieee.org/document/8990979, maximum expected acc is 70% using resnet18 and kaggle dataset